In [1]:
import re
import pandas as pd
import os

In [2]:
os.getcwd()

'C:\\Users\\MI24133\\Downloads'

In [3]:
# Parametros: carpeta donde están los archivos del código y Nombre del Proyecto

dir_path = "BancaMayorista/"
proceso = "Banca Mayorista"

In [4]:
## Se leen los distintos archivos del directorio especificado, y se pegan

code = ""
print("Archivos leídos:")
for filename in os.listdir(dir_path):
    if ( filename[-3:] == 'txt') or ( filename[-3:] == 'sas'):
        with open(dir_path + filename, "rb") as my_file:
            print(dir_path + filename)
            code = code + my_file.read().decode("ANSI")#.decode("UTF-8")
            
# Limpieza de string (código)
code = code.upper()
code = code.replace('\n', ' ')
code = code.replace('\r', ' ')

# Eliminar comentarios (código delimitado por "/*" y "*/")
code_clean = re.sub(r'\/\*[\s\S]*?\*\/', "", code)

Archivos leídos:
BancaMayorista/0001_carga_parametros.txt
BancaMayorista/0001_cartera_hubycorner.txt
BancaMayorista/0002_carga_basedeofuscados.txt
BancaMayorista/0003_carga_base_invitaciones.txt
BancaMayorista/0004_formatoabasede_invitaciones.txt
BancaMayorista/0005_solo_mismo_banquero.txt
BancaMayorista/0006_data_dummy.txt
BancaMayorista/0007_pegando_datos.txt
BancaMayorista/0008_carga_bases_lar.txt
BancaMayorista/0009_pega_datos2.txt
BancaMayorista/0010_pega_datos3.txt
BancaMayorista/0011_elige_muestras.txt


In [5]:
# Reemplazar macro variables (definidas como: LET XXX) 

read0 = """\%LET\s*(\w+)\s\=\s?([A-Z0-9/'"]+)"""

macro_vars = set(re.findall(read0, code_clean)) 
for x in macro_vars:
    code_clean = code_clean.replace('&' + x[0] + '.', x[1])

print("Macro variables:")
macro_vars

Macro variables:


{('ANIO', '2023'),
 ('CUAREN_DETR', '90'),
 ('CUAREN_PASI', '90'),
 ('CUAREN_PROM', '90'),
 ('EXPORTA', '20230908'),
 ('FECHAPLANTILLA', '31AUG2023'),
 ('FECHA_ENVIO', "'08/09/2023'"),
 ('FH_CIERRE', "'31AUG2023"),
 ('FH_ENVIO', "'08SEP2023'D"),
 ('FH_FIN', "'31AUG2023'D"),
 ('FH_INI', "'01AUG2023'D"),
 ('FH_INI_SEM', "'01SEP2022'D"),
 ('LEVANTAMIENTO', "'08/09/2023'"),
 ('MES', 'SEP2023'),
 ('MES2', '"SEPTIEMBRE"'),
 ('MESLISTA1', "'JUNIO'"),
 ('MESLISTA2', "'JULIO'"),
 ('MESLISTA3', "'AGOSTO'"),
 ('MESPLANT', 'AGO23'),
 ('MES_ENVIO', "'SEPTIEMBRE'"),
 ('MES_ENVIO', 'SEP23'),
 ('MES_ENVIO2', 'SEP2023'),
 ('PATHM', 'BLPBP001')}

In [6]:
## Patrones con regex para tablas que se leen y se escriben

read1 = "FROM\s*(?!WORK\.)(\w+)?\.(\w+)"
read2 = "JOIN\s*(?!WORK\.)(\w+)?\.(\w+)"
read3 = "SET\s*(?!WORK\.)(\w+)?\.(\w+)"
read4 = """INFILE\s*=\s*["][A-Z0-9/]*/(\w+).(\w+)"""

write1 = "CREATE TABLE\s*(?!WORK\.)(\w+)?\.(\w+)"
write2 = "DATA\s*(?!WORK\.)(\w+)?\.(\w+)"
write3 = """OUTFILE\s*=\s*["][A-Z0-9/]*/(\w+).(\w+)"""

# Buscar matches dentro del código
read_tabl = set(re.findall(read1, code_clean) + 
                re.findall(read2, code_clean) + 
                re.findall(read3, code_clean) + 
                [(x[1], x[0]) for x in re.findall(read4, code_clean)] )

write_tabl = set(re.findall(write1, code_clean) +
                 re.findall(write2, code_clean) + 
                 [(x[1], x[0]) for x in re.findall(write3, code_clean)] )

len(read_tabl)

28

In [7]:
# Determinación de Tablas Intermedias
r_tables = read_tabl - write_tabl
w_tables = write_tabl - read_tabl
i_tables = read_tabl & write_tabl

i_tables

{('ENCBPYP', 'ENVIADOS_PYP'),
 ('ENCBPYP', 'HIST_HUBCORNER'),
 ('ENCBPYP', 'INV_FILE2_SEP2023'),
 ('ENCBPYP', 'PEGA_DATOS_INVIT_SEP2023'),
 ('ENCBPYP', 'PRUEBA_FINAL2_'),
 ('ENCBPYP', 'SOLO_MISMO_BANQUERO_SEP2023_2'),
 ('ENCBPYP', 'TU_LDL_BASE_INS_LALO_SORTED'),
 ('ENCBPYP2', 'ENTREGA_BPYP_SEP2023_FINALFIN2'),
 ('LARI', 'TU_LDL_BPYP_SEP2023_ENVIOFINAN')}

In [8]:
# Se crea un DataFrame con todas la tablas
r_tables =  pd.DataFrame(r_tables, columns=['Schema', 'Tabla'])
r_tables["Tipo"] = "Input"

i_tables =  pd.DataFrame(i_tables, columns=['Schema', 'Tabla'])
i_tables["Tipo"] = "Intermedia"

w_tables =  pd.DataFrame(w_tables, columns=['Schema', 'Tabla'])
w_tables["Tipo"] = "Output"

tables = pd.concat([r_tables, i_tables, w_tables])
tables

,Schema,Tabla,Tipo
0,DMC,PAT_CARTERA_MENSUAL,Input
1,EVA,TU_JETB_NO_ENCUESTAR_1,Input
2,ENCBPYP,CLIENTES_MALHUBAGO2022,Input
3,EVA,TU_JETB_CATALOGO_EDO_OFICINA,Input
4,ENCBPYP,PLANTILLA_FINANZAS_AGO23,Input
5,DMC,PAT_CARTERA_DIA_UNICOS,Input
6,KLARMXPV,TLA1048_CTO_FIDE_BO,Input
7,EVA,TU_JETB_CLIENTES_EXP,Input
8,DMC,RP_08_TPATRIMONIAL_015_TOTAL_,Input
9,KLARMXPV,TLA543_PER_CTE_BASICO_SEM,Input


In [9]:
## Matches con regex para info librerías

read_libraries = """LIBNAME\s+(\w+)\s+([A-Z]*)\"?\'?\/?([A-Z0-9/*]*)?"""

read_lib = re.findall(read_libraries, code_clean)
libraries = pd.DataFrame(read_lib, columns=['Base', 'Sistema', 'Path'])
libraries = libraries.drop_duplicates()
libraries.loc[libraries['Sistema'] == '', 'Sistema'] = 'SAS'
libraries


,Base,Sistema,Path
0,DMC,ORACLE,
1,LARI,SAS,CXAT/GABRIELB/EVA/ENCUESTAS/BPYP
2,DATSEN,SAS,CXAT/DATSEN
3,ENCBPYP,SAS,CXAT/GABRIELB/EVA/ENCUESTAS/BPYP
4,ENCBPYP2,SAS,CXAT/GABRIELB/EVA/ENCUESTAS/BPYP2
5,EVA,SAS,CXAT/GABRIELB/EVA
6,LAP,ORACLE,
7,KLARMXPV,TERADATA,
8,PAC,TERADATA,


In [10]:
# Juntar con Información de Bases
full_tabl = tables.merge(libraries, left_on='Schema', right_on='Base', how='left')
full_tabl["Proceso"] = proceso
full_tabl[['Sistema']] = full_tabl[['Sistema']].fillna('Manual')

full_tabl

,Schema,Tabla,Tipo,Base,Sistema,Path,Proceso
0,DMC,PAT_CARTERA_MENSUAL,Input,DMC,ORACLE,,Banca Mayorista
1,EVA,TU_JETB_NO_ENCUESTAR_1,Input,EVA,SAS,CXAT/GABRIELB/EVA,Banca Mayorista
2,ENCBPYP,CLIENTES_MALHUBAGO2022,Input,ENCBPYP,SAS,CXAT/GABRIELB/EVA/ENCUESTAS/BPYP,Banca Mayorista
3,EVA,TU_JETB_CATALOGO_EDO_OFICINA,Input,EVA,SAS,CXAT/GABRIELB/EVA,Banca Mayorista
4,ENCBPYP,PLANTILLA_FINANZAS_AGO23,Input,ENCBPYP,SAS,CXAT/GABRIELB/EVA/ENCUESTAS/BPYP,Banca Mayorista
5,DMC,PAT_CARTERA_DIA_UNICOS,Input,DMC,ORACLE,,Banca Mayorista
6,KLARMXPV,TLA1048_CTO_FIDE_BO,Input,KLARMXPV,TERADATA,,Banca Mayorista
7,EVA,TU_JETB_CLIENTES_EXP,Input,EVA,SAS,CXAT/GABRIELB/EVA,Banca Mayorista
8,DMC,RP_08_TPATRIMONIAL_015_TOTAL_,Input,DMC,ORACLE,,Banca Mayorista
9,KLARMXPV,TLA543_PER_CTE_BASICO_SEM,Input,KLARMXPV,TERADATA,,Banca Mayorista
